## Packages

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from gensim.models import Word2Vec
import numpy as np



## Functions

In [11]:
def createVocabulary(corpus):
    """
    - Cria vocab (palavra e sua respectiva frequência no corpus)
    - Cria tokens (palavras do corpus)

    :param corpus: lista de string
    :return vocab, tokens, vocab_size:
    """
    tokens = []  # {'deeds', 'old', ...} 71666
    vocab = {}  # {'deeds': 2, 'old': 20', ...} 17971
    for text in corpus:
        for token in text.split():
            tokens.append(token)
            if token in vocab:
                vocab[token] += 1
            else:
                vocab[token] = 1
    vocab_size = len(vocab)
    return tokens, vocab, vocab_size

In [19]:
def findMaxLen(sequence):
    max_len = 0
    for text in sequence:
        if len(text) > max_len:
            max_len = len(text)
    return max_len

def findAverageLen(sequence):
    total_len = 0
    for text in sequence:
        total_len += len(text)
    return total_len / len(sequence)

## Load data

In [2]:
df_pp = pd.read_csv('../../assets/df_pp.csv')
df_pp

,title,tags,link,label
0,dividendos as <NUM> acoes preferidas <NUM> ana...,"['Carteira Recomendada', 'Comprar ou vender?',...",https://www.moneytimes.com.br/dividendos-as-10...,1
1,itau jhsf banco brasil confira agenda resultad...,"['Alper (APER3)', 'balanços trimestrais', 'Bol...",https://www.suno.com.br/noticias/itau-jhsf-ban...,2
2,destaques empresas petrobras petr4 jbs jbss3 h...,"['Arezzo (ARZZ3)', 'Cia Hering (HGTX3)', 'dest...",https://www.suno.com.br/noticias/destaques-de-...,0
3,itau unibanco itub4 vai pagar proventos comeco...,"['Bancos', 'Dividendos', 'Empresas', 'Itaú Uni...",https://www.moneytimes.com.br/itau-unibanco-it...,2
4,petrobras petr4 elege novo conselho maior part...,"['Ações', 'Adriano Pires', 'Combustíveis', 'Em...",https://www.moneytimes.com.br/petrobras-petr4-...,0
...,...,...,...,...
11951,petrobras guia nova maxima ibovespa sessao meg...,"['Ações', 'Empresas', 'Ibovespa', 'Leilão', 'M...",https://www.moneytimes.com.br/petrobras-guia-n...,0
11952,petrobras estende vencimento emprestimo banco ...,"['Empresas', 'Petrobras']",https://www.moneytimes.com.br/petrobras-estend...,0
11953,petrobras percepcao risco sobre politica preco...,"['Ações', 'Combustíveis', 'Commodities', 'Comp...",https://www.moneytimes.com.br/petrobras-percep...,0
11954,bndes espera vender cerca <NUM> bi debentures ...,"['BNDES', 'BNDESPar', 'debêntures', 'Vale (VAL...",https://www.suno.com.br/noticias/bndes-venda-d...,1


## Splitting Dataset

In [3]:
X = df_pp.title
y = df_pp.label

In [6]:
# Split data into train and test
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8, random_state=52)  # 80% train

X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=52)  # 10% val, 10% testc

In [8]:
print(y_train.value_counts())
print(y_val.value_counts())
print(y_test.value_counts())

0    4877
1    3007
2    1680
Name: label, dtype: int64
0    646
1    349
2    201
Name: label, dtype: int64
0    590
1    384
2    222
Name: label, dtype: int64


## Tokenization, Vocabulary

In [13]:
tokens, vocab, vocab_size = createVocabulary(X_train)
len(vocab)

8010

## Numericalization

In [22]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(X_train)

## Padding

In [23]:
max_len = findMaxLen(train_sequences)
max_len = int(max_len/2)

In [29]:
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')

val_sequences = tokenizer.texts_to_sequences(X_val)
val_padded = pad_sequences(val_sequences, maxlen=max_len, padding='post', truncating='post')

## Load word2vec model

In [33]:
model = Word2Vec.load('../../assets/word2vec.model')

model.wv.most_similar('petrobras')

[('petr4', 0.9948422312736511),
 ('diz', 0.9689343571662903),
 ('combustiveis', 0.9598903656005859),
 ('bolsonaro', 0.9582469463348389),
 ('intervir', 0.9552468657493591),
 ('sal', 0.9517554640769958),
 ('gas', 0.9510222673416138),
 ('venda', 0.9506813883781433),
 ('natural', 0.9480621814727783),
 ('premenos', 0.9469091296195984)]

In [34]:
# List of nparrays of size 300
embeddings_dict = {}
for i, word in enumerate(model.wv.index_to_key):
    embeddings_dict[word] = model.wv[word]

In [38]:
# create matrix with vocab train words
embeddings_matrix = np.zeros((len(word_index),300))
for word, i in word_index.items():
    embeddings_vector = embeddings_dict.get(word)
    if embeddings_vector is not None:
        embeddings_matrix[i-1] = embeddings_vector

## RNN Model

### Packages

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.losses import BinaryCrossentropy

from keras.optimizers import Adam
import matplotlib.pyplot as plt

### Functions

In [41]:
def index2word(word_index):
    index_word = {}
    for key in word_index:
        index_word[word_index[key]] = key
    return index_word


def seq2text(seq, index_word):
    text = []
    for index in seq:
        text.append(index_word[index])
    return text

def show_confusion_matrix(cm):
        print("Confusion Matrix")
        plt.figure(figsize=(10, 7))

        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'],
                    yticklabels=['Negative', 'Positive'])
        plt.xlabel('Predicted')
        plt.ylabel('Truth')
        plt.title('Confusion Matrix')
        plt.show()

### Declaration

In [42]:
model = Sequential([
    Embedding(input_dim=vocab_size+1, output_dim= 300,input_length=max_len, trainable=False, weights=[embeddings_matrix]),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.6),
    # Bidirectional(LSTM(hp.Choice('units',[32,64]))),
    Bidirectional(LSTM(256)),
    Dense(32, activation='relu'),
    # Dropout(0.6),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 300)           2403300   
                                                                 
 bidirectional (Bidirectiona  (None, 11, 512)          1140736   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 512)           0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dense (Dense)               (None, 32)                16416     
                                                                 
 dense_1 (Dense)             (None, 1)                 3

### Training

In [ ]:
loss = BinaryCrossentropy(from_logits=False)
optimizer = Adam(0.0001)
metrics = ['accuracy']
model.compile(loss=loss,optimizer=optimizer,metrics=metrics)

history = model.fit(train_padded, y_train, epochs=30, validation_data=(val_padded,y_val), verbose=2)

Epoch 1/30
299/299 - 109s - loss: -6.4533e-01 - accuracy: 0.5332 - val_loss: -1.7918e+00 - val_accuracy: 0.6555 - 109s/epoch - 366ms/step
Epoch 2/30


### Evaluation

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(history.history['accuracy'])
ax1.plot(history.history['val_accuracy'])
ax1.set_title('Model Accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Validation'], loc='upper left')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('Model Loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Validation'], loc='upper left')
plt.show()

### Predictions

In [ ]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

In [ ]:
index_word = index2word(word_index)

In [ ]:
pred_prob = model.predict(test_padded)


In [ ]:
y_pred = [1 if p > 0.5 else 0 for p in pred_prob]


In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
df_results = pd.DataFrame()
X_test = list(X_test)
y_test = list(y_test)
df_results['sequence'] = test_sequences
df_results['X_test'] = X_test
df_results['seq2text'] = df_results['sequence'].apply(lambda x: seq2text(x, index_word))
df_results['y_pred'] = y_pred
df_results['y_true'] = y_test
df_results

### Metrics

In [ ]:
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1score = f1_score(y_test,y_pred)
cm = confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))
show_confusion_matrix(cm)